In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras import layers, Model, Sequential
from tensorflow.keras.layers import Input, Dense

In [2]:
base_dir = 'dogs_breed'
valid_dir = 'valid'
train_dir = 'train'
train_path = os.path.join(base_dir, train_dir)
valid_path = os.path.join(base_dir, valid_dir)
print(train_path)

dogs_breed\train


In [3]:
model_VGG16 = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)
model_VGG16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
personal_model = Sequential()
for layer in model_VGG16.layers[:-1]:
    personal_model.add(layer)
    
personal_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [5]:
personal_model.add(Dense(units=len(os.listdir(train_path)), activation='softmax'))
personal_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [6]:
personal_model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
train_datagen_personal_vgg = ImageDataGenerator(rescale=1./255)
  
val_datagen_personal_vgg = ImageDataGenerator(rescale=1./255)

train_img_nb = 16469
valid_img_nb = 4111
batch_size = 20
epochs = 1

train_generator = train_datagen_personal_vgg.flow_from_directory(
    train_dir,  
    target_size=(224, 224),  
    batch_size=batch_size,
    classes=os.listdir(train_path)
)

valid_generator = val_datagen_personal_vgg.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    classes=os.listdir(train_path)
)

history = personal_model.fit(
    train_generator,
    steps_per_epoch=int(train_img_nb / batch_size),
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=int(valid_img_nb / batch_size),
)

Found 11574 images belonging to 120 classes.
Found 4895 images belonging to 120 classes.
Epoch 1/15
722/722 - 111s - loss: 4.7082 - acc: 0.3000 - val_loss: 4.6678 - val_acc: 0.2052
Epoch 2/15
722/722 - 111s - loss: 2.6021 - acc: 0.5000 - val_loss: 4.7205 - val_acc: 0.2041
Epoch 3/15
722/722 - 110s - loss: 2.4501 - acc: 0.4000 - val_loss: 4.6149 - val_acc: 0.2072
Epoch 4/15
722/722 - 111s - loss: 2.8080 - acc: 0.5000 - val_loss: 4.5796 - val_acc: 0.2033
Epoch 5/15
722/722 - 111s - loss: 3.3972 - acc: 0.2500 - val_loss: 4.7458 - val_acc: 0.1961
Epoch 6/15
722/722 - 110s - loss: 2.0587 - acc: 0.4000 - val_loss: 5.0993 - val_acc: 0.1884
Epoch 7/15
722/722 - 111s - loss: 2.1390 - acc: 0.5000 - val_loss: 5.1103 - val_acc: 0.1925
Epoch 8/15
722/722 - 111s - loss: 3.2941 - acc: 0.2000 - val_loss: 4.6682 - val_acc: 0.2087
Epoch 9/15
722/722 - 111s - loss: 2.8543 - acc: 0.3500 - val_loss: 4.9082 - val_acc: 0.2016
Epoch 10/15
722/722 - 110s - loss: 2.8218 - acc: 0.4000 - val_loss: 4.9528 - val_ac